In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from time import time

2022-05-03 08:46:16.572661: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-03 08:46:16.572751: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# tensor =tf.constant([[1,2,3],[4,5,6]])

# tensor.numpy()

In [3]:
df_pagamentos= pd.read_csv('/home/alessandro/tcc/dados/payment_data.csv')
df_pagamentos.set_index('id', inplace=True) # seta o id como index
df_consumidores = pd.read_csv('/home/alessandro/tcc/dados/customer_data.csv')
df_consumidores.set_index('id', inplace=True) # seta o id como index

In [4]:
dados_consumidores=df_consumidores.join(df_pagamentos, how='inner') # junta os dois dataframes
dados_consumidores.columns

Index(['label', 'fea_1', 'fea_2', 'fea_3', 'fea_4', 'fea_5', 'fea_6', 'fea_7',
       'fea_8', 'fea_9', 'fea_10', 'fea_11', 'OVD_t1', 'OVD_t2', 'OVD_t3',
       'OVD_sum', 'pay_normal', 'prod_code', 'prod_limit', 'update_date',
       'new_balance', 'highest_balance', 'report_date'],
      dtype='object')

In [5]:
colunas=['alvo',
 'recurso_1',
 'recurso_2',
 'recurso_3',
 'recurso_4',
 'recurso_5',
 'recurso_6',
 'recurso_7',
 'recurso_8',
 'recurso_9',
 'recurso_10',
 'recurso_11',
 'atraso_t1',
 'atraso_t2',
 'atraso_t3',
 'atraso_total',
 'pagemento_normal',
 'codigo_produto',
 'limite_credito',
 'data_atualizada',
 'saldo_atual_produto',
 'saldo_mais_alto_produto',
 'report_date']


dados_consumidores.columns=colunas
dados_consumidores.to_excel("")

,alvo,recurso_1,recurso_2,recurso_3,recurso_4,recurso_5,recurso_6,recurso_7,recurso_8,recurso_9,...,atraso_t2,atraso_t3,atraso_total,pagemento_normal,codigo_produto,limite_credito,data_atualizada,new_balance,highest_balance,report_date
id,,,,,,,,,,,,,,,,,,,,,
54982353,0,1,1130.0,2,1000000.0,2,4,-1,100,5,...,0,0,0,1,10,55000.0,27/08/2014,0.0,2068.0,12/06/2014
54982353,0,1,1130.0,2,1000000.0,2,4,-1,100,5,...,0,0,1,31,10,550000.0,03/09/2013,326684.4,609683.0,18/12/2015
54982353,0,1,1130.0,2,1000000.0,2,4,-1,100,5,...,0,0,0,19,10,NaN,16/07/2011,31677.6,204037.0,14/12/2015
54982353,0,1,1130.0,2,1000000.0,2,4,-1,100,5,...,0,35,31500,0,10,12100.0,27/12/2008,12142.8,10619.0,14/07/2009
54982353,0,1,1130.0,2,1000000.0,2,4,-1,100,5,...,0,0,0,26,10,660000.0,12/03/2007,252998.4,775030.0,23/12/2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59006219,0,4,NaN,2,111000.0,2,8,5,110,4,...,0,0,28,35,10,33000.0,28/01/2013,9956.4,22591.0,04/02/2016
59006219,0,4,NaN,2,111000.0,2,8,5,110,4,...,0,0,0,36,10,NaN,27/01/2013,1344.0,44655.0,13/03/2016
59006239,0,7,1322.0,3,68000.0,2,11,5,86,3,...,0,0,0,12,10,NaN,23/03/2015,2492.4,15440.0,19/02/2016


In [ ]:
dados_consumidores['alvo'].value_counts() # conta quantas vezes cada label aparece

0    6863
1    1387
Name: alvo, dtype: int64

In [7]:
dados_consumidores.isnull().sum() # conta quantos valores faltantes

alvo                   0
recurso_1              0
recurso_2           1028
recurso_3              0
recurso_4              0
recurso_5              0
recurso_6              0
recurso_7              0
recurso_8              0
recurso_9              0
recurso_10             0
recurso_11             0
atraso_t1              0
atraso_t2              0
atraso_t3              0
atraso_total           0
pagemento_normal       0
codigo_produto         0
limite_credito      6118
data_atualizada       26
new_balance            0
highest_balance      409
report_date         1114
dtype: int64

In [8]:
dados_consumidores['codigo_produto'].value_counts() # conta quantas vezes cada categoria aparece

10    4523
6     1144
5      962
1      427
13     425
2      239
0      147
7      147
12      56
19      35
8       24
24      24
17      22
15      22
3       21
26      14
25       5
4        4
27       3
22       3
9        3
Name: codigo_produto, dtype: int64

In [9]:
for prod_code in dados_consumidores["codigo_produto"].unique():
    
    dados_consumidores["codigo_produto{}".format(prod_code)] = dados_consumidores["codigo_produto"] == prod_code
    
dados_consumidores



,alvo,recurso_1,recurso_2,recurso_3,recurso_4,recurso_5,recurso_6,recurso_7,recurso_8,recurso_9,...,codigo_produto22,codigo_produto8,codigo_produto19,codigo_produto24,codigo_produto15,codigo_produto3,codigo_produto25,codigo_produto4,codigo_produto26,codigo_produto9
id,,,,,,,,,,,,,,,,,,,,,
54982353,0,1,1130.0,2,1000000.0,2,4,-1,100,5,...,False,False,False,False,False,False,False,False,False,False
54982353,0,1,1130.0,2,1000000.0,2,4,-1,100,5,...,False,False,False,False,False,False,False,False,False,False
54982353,0,1,1130.0,2,1000000.0,2,4,-1,100,5,...,False,False,False,False,False,False,False,False,False,False
54982353,0,1,1130.0,2,1000000.0,2,4,-1,100,5,...,False,False,False,False,False,False,False,False,False,False
54982353,0,1,1130.0,2,1000000.0,2,4,-1,100,5,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59006219,0,4,NaN,2,111000.0,2,8,5,110,4,...,False,False,False,False,False,False,False,False,False,False
59006219,0,4,NaN,2,111000.0,2,8,5,110,4,...,False,False,False,False,False,False,False,False,False,False
59006239,0,7,1322.0,3,68000.0,2,11,5,86,3,...,False,False,False,False,False,False,False,False,False,False


In [10]:
dados_consumidores['recurso_4'].unique()

array([1000000., 1200000.,  286000.,   70000.,  120000.,   30000.,
        325000.,   77000.,  130000.,   90000.,  157000.,  154000.,
        102000.,  163000.,   88000.,  132000.,   82000.,  117000.,
         58000.,   78000.,  270000.,   72000.,   89000.,  114000.,
        180000.,  220000.,   56000.,   35000.,  127000.,  104000.,
         47000.,  143000.,  195000.,  106000.,   96000.,  125000.,
         94000.,   59000.,  110000.,   54000.,   76000.,   83000.,
         57000.,  144000.,   79000.,  134000.,   51000.,   81000.,
        112000.,   80000.,  155000.,   91000.,   61000.,  111000.,
        100000.,  167000.,   60000.,   15000.,   50000.,  227000.,
         66000.,   68000.,   71000.,  121000.,   63000.,  193000.,
        187000.,  183000.,   97000.,  290000.,  115000.,  150000.,
         93000.,  140000.,  162000.,   98000.,  203000.,  149000.,
        312000.,  264000.,  108000.,  243000.,  500000.,   73000.,
        116000.,  379000.,  173000.,  137000.,  263000.,  1380

In [11]:
for feature_id in [1, 3, 5, 6, 7, 9]:
    for value in dados_consumidores["recurso_{}".format(feature_id)].unique():
        dados_consumidores["recurso_{}_{}".format(feature_id, value)] = dados_consumidores["recurso_{}".format(feature_id)] == value
dados_consumidores


,alvo,recurso_1,recurso_2,recurso_3,recurso_4,recurso_5,recurso_6,recurso_7,recurso_8,recurso_9,...,recurso_7_10,recurso_7_8,recurso_7_3,recurso_7_4,recurso_7_1,recurso_9_5,recurso_9_3,recurso_9_4,recurso_9_1,recurso_9_2
id,,,,,,,,,,,,,,,,,,,,,
54982353,0,1,1130.0,2,1000000.0,2,4,-1,100,5,...,False,False,False,False,False,True,False,False,False,False
54982353,0,1,1130.0,2,1000000.0,2,4,-1,100,5,...,False,False,False,False,False,True,False,False,False,False
54982353,0,1,1130.0,2,1000000.0,2,4,-1,100,5,...,False,False,False,False,False,True,False,False,False,False
54982353,0,1,1130.0,2,1000000.0,2,4,-1,100,5,...,False,False,False,False,False,True,False,False,False,False
54982353,0,1,1130.0,2,1000000.0,2,4,-1,100,5,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59006219,0,4,NaN,2,111000.0,2,8,5,110,4,...,False,False,False,False,False,False,False,True,False,False
59006219,0,4,NaN,2,111000.0,2,8,5,110,4,...,False,False,False,False,False,False,False,True,False,False
59006239,0,7,1322.0,3,68000.0,2,11,5,86,3,...,False,False,False,False,False,False,True,False,False,False


In [12]:
dados_consumidores = dados_consumidores.drop(["limite_credito", "report_date", "data_atualizada", "codigo_produto", "recurso_1", "recurso_3", "recurso_5", "recurso_6", "recurso_7", "recurso_9"], axis=1)

In [13]:
dados_consumidores_array_np = dados_consumidores.to_numpy(na_value=np.nan).astype(float)

In [14]:


#imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

imputer = SimpleImputer(verbose=2)
data = imputer.fit_transform(dados_consumidores_array_np)

# (dados_consumidores==np.nan).any()
# print(imp_mean)

In [15]:
scaler = MinMaxScaler()

dados_consumidores_normalizado =scaler.fit_transform(data)

dados_teste =scaler.fit_transform(data)


In [16]:
# balanceado o base de dados
mask = dados_consumidores_normalizado[:, 0] == 1
risk_customers = dados_consumidores_normalizado[mask]
print("numero de clientes com risco de crédito alto = {}".format(risk_customers.shape[0]))

safe_customers = dados_consumidores_normalizado[~mask]
indices = np.random.choice(safe_customers.shape[0], risk_customers.shape[0]*2, replace=False)
safe_customers = safe_customers[indices]
print("numero de clientes com risco de crédito baixo = {}".format(safe_customers.shape[0]))

dados_consumidores_normalizado = np.concatenate([risk_customers, safe_customers], axis=0)
print("tamanho do novo dataframe = ", dados_consumidores_normalizado.shape)

numero de clientes com risco de crédito alto = 1387
numero de clientes com risco de crédito baixo = 2774
tamanho do novo dataframe =  (4161, 70)


In [17]:

X=dados_consumidores_normalizado[:,1:]  # features
y=dados_consumidores_normalizado[:,0] #label 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=10)



In [18]:

def create_model(optimizer='adam'):

    model =tf.keras.models.Sequential()

    model.add(tf.keras.layers.Dense(units=64, input_shape=(69,)))
    model.add(tf.keras.layers.Dense(36, activation='relu'))
    model.add(tf.keras.layers.Dense(18, activation='relu'))
    model.add(tf.keras.layers.Dense(9, activation='relu'))

    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


    model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy'])

    return model

model=create_model()
# model = KerasClassifier(build_fn=create_model, verbose=1)

# optimizers = ['rmsprop','adam']
# epochs = np.array([10])
# batches = np.array([32])

# param_grid= dict(optimizer=optimizers, epochs=epochs, batch_size=batches)
# grid= GridSearchCV(estimator=model, param_grid=param_grid)

# start=time()
# grid_result = grid.fit(X_train, y_train)
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# for params, mean_score, scores in grid_result.grid_scores_:
#     print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))
# print("total time:",time()-start)

2022-05-03 08:46:48.511109: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-03 08:46:48.511203: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-03 08:46:48.511287: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (alessandro-Lenovo-YOGA-520-14IKB): /proc/driver/nvidia/version does not exist
2022-05-03 08:46:48.543930: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
model.fit(X_train, y_train, epochs=500, batch_size=32, verbose=0)
model.evaluate(X_test, y_test, verbose=0)

[1.0001740455627441, 0.8959243297576904]

In [ ]:


yk=model.predict(X_test)
score=model.evaluate(X_test, y_test)
resposta=np.array([1 if yk[i]>0.5 else 0 for i in range(len(yk))])
bons_pagadores=resposta==0 

print("Bom pagador",resposta[bons_pagadores].shape[0])
print("Mau pagador",resposta[~bons_pagadores].shape[0])


resp=np.array(resposta).reshape(-1,1)
alvo=np.array(y_test).reshape(-1,1)
comparativo=np.append(alvo,resp,axis=1)


df_saida=pd.DataFrame(comparativo, columns=['alvo','previsto'])

# df=pd.DataFrame(data=resposta, index=y_test, columns=['label'])
# df.loc[df['label']==1,'label']='Mau pagador'
# df.loc[df['label']==0,'label']='Bom pagador'
df_saida.to_excel('resultado.xlsx', index=False)

43/43 [==============================] - 0s 2ms/step - loss: 0.3548 - accuracy: 0.9520
Bom pagador 902
Mau pagador 472


In [ ]:
# # balance the dataset
# mask = dados_teste[:, 0] == 1

# risk_customers = dados_teste[mask]
# print("number of risk customers = {}".format(risk_customers.shape[0]))

# safe_customers = dados_teste[~mask]
# indices = np.random.choice(safe_customers.shape[0], risk_customers.shape[0]*2, replace=False)

# safe_customers = safe_customers[indices]
# print("number of safe customers = {}".format(safe_customers.shape[0]))

# data = np.concatenate([risk_customers, safe_customers], axis=0)
# print("Shape of selected data = ", dados_teste.shape)